In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_wcs(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
			)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__

live = client.is_live()
connected = client.is_connected()

print(f"Weaviate client is ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"live: {live}")
print(f"Connected: {connected}")

In [ ]:
# Trigger Read Repairs consistency level to ALL
import requests

def main():
    base_url = "<ENDPOINT>"
    class_name = "<COLL or MAKE a LIST>"
    bearer_token = "<API KEY>"

    headers = {
        "Authorization": f"Bearer {bearer_token}"
    }

    # Step 1: Fetch all UUIDs for a class
    limit = 500
    offset = 0
    all_uuids = []

    print(f"=== Fetching all objects for class '{class_name}' ===")
    while True:
        params_list = {
            "limit": limit,
            "offset": offset,
            "class": class_name 
        }
        resp = requests.get(f"{base_url}/v1/objects", params=params_list, headers=headers)

        if resp.status_code != 200:
            print(f"Error listing objects for '{class_name}': {resp.status_code} {resp.text}")
            break

        data = resp.json()
        objects_batch = data.get("objects", [])
        if not objects_batch:
            break

        for i, obj in enumerate(objects_batch, start=offset):
            uuid = obj.get("id")
            all_uuids.append(uuid)
            print(f"Found object #{i}: {uuid}")

        offset += limit

    print(f"\nFetched {len(all_uuids)} total objects in class '{class_name}'.\n")

    # Step 2: Fetch each UUID with ?consistency_level=ALL
    print(f"=== Checking objects for class '{class_name}' ===")
    for index, uuid in enumerate(all_uuids):
        url = f"{base_url}/v1/objects/{class_name}/{uuid}"
        params_single = {
            "consistency_level": "ALL"
        }
        resp_single = requests.get(url, params=params_single, headers=headers)

        if resp_single.status_code == 200:
            obj_data = resp_single.json()
            name_val = obj_data.get("properties", {}).get("name")
            print(f"[{index}] UUID={uuid} => name={name_val}")
        elif resp_single.status_code == 404:
            print(f"[{index}] UUID={uuid} => Not found.")
        else:
            print(f"[{index}] UUID={uuid} => Error {resp_single.status_code}: {resp_single.text}")

if __name__ == "__main__":
    main()
